# Conduit Transformers

The conduit transformers library is the original framework for making power engineering computations and can be used for fast and efficient one-off computations on an individual stream or set of streams. These functions are necessarily more manual than the graph based method of computation, but allows you to work a bit more quickly than you might with the data flow. However, it is strongly recommended that you use the graph based computation framework wherever possible!

The transformer API is as follows:


```python

def transformer(data, tags, annotations):
    return data, tags, annotations
```

All transformers take data as either a `pd.Series` or `pd.DataFrame` and tags and annotations either as dictionaries with the tags and annotations for a single stream, or a list of dictionaries if there are multiple streams in a data frame. The transformer then returns the computed data along with new tags and annotations that describe the returned data stream or streams.

In [1]:
import btrdb
import pandas as pd

from conduit.constants import *
from btrdb.utils.timez import ns_delta
from conduit.transformers.phasor import *
from conduit.stream import points_to_series
from conduit.transformers.conversions import * 

In [2]:
db = btrdb.connect(profile="dominion.predictivegrid.com")
db.info()

{'majorVersion': 5, 'build': '5.6.3', 'proxy': {'proxyEndpoints': []}}

## Data Selection and Materialization

Conduit transformers require you to materialize your data into a `pd.Series` or `pd.DataFrame` before you can begin to work with them. To help with this, you can use the `conduit.stream.points_to_series` function to convert a single stream query into a time-indexed pandas data structure. This function can handle either raw values queries or windows queries that return statistical aggregate points. 

### Values to Series

Convert a `stream.values()` query to a `pd.Series` as follows:

In [3]:
stream = db.stream_from_uuid("11ed9a21-4371-5aa1-8085-b990869d0b51") # Northeast Carver 3 Ean VPHM
series = points_to_series(
            stream.values("2019-09-25T12:00:00.000", "2019-09-25T12:01:00.000"),
            name=str(stream.uuid),
         )
series.head()

2019-09-25 12:00:00.000000000    67837.398438
2019-09-25 12:00:00.033333100    67878.062500
2019-09-25 12:00:00.066666300    67845.203125
2019-09-25 12:00:00.100000200    67860.359375
2019-09-25 12:00:00.133333200    67881.507812
Name: 11ed9a21-4371-5aa1-8085-b990869d0b51, dtype: float64

Note that you can pass the result of the `stream.values()` query directly to `points_to_series` along with other arguments such as `name` and `dtype` that add metadata to the series object. Most commonly, we simply pass the stream UUID to the `name` argument to easily identify it, but you can pass whatever identifying label you wish. 

Note also that the timestamps are automatically converted into `np.datetime64` objects, which allow you to do easy time series analysis on the data stream. For more on these arguments, use `help(points_to_series)`.

### Windows and Aligned Windows to Series 

The `points_to_series` function can also convert queries that return statistical points such as `stream.windows` and `stream.aligned_windows`. To do this you need to specify which statistical aggregate you would like returned, which is by default the `mean`. 

In [4]:
series = points_to_series(
            stream.windows("2019-09-25T12:00:00.000", "2019-09-25T12:01:00.000", ns_delta(seconds=5), depth=30),
            name="{} (mean)".format(stream.uuid)
         )
series.head()

2019-09-25 12:00:00    67909.286458
2019-09-25 12:00:05    67907.396560
2019-09-25 12:00:10    67861.438227
2019-09-25 12:00:15    67841.727961
2019-09-25 12:00:20    67973.322553
Name: 11ed9a21-4371-5aa1-8085-b990869d0b51 (mean), dtype: float64

If you would like the number of points returned, you could use `count` as follows:

In [5]:
series = points_to_series(
            stream.aligned_windows("2019-09-25T12:00:00.000", "2019-09-25T12:01:00.000", pointwidth=32),
            agg="count", name="{} (count)".format(stream.uuid)
         )
series.head()

2019-09-25 11:59:58.356652032    129
2019-09-25 12:00:02.651619328    129
2019-09-25 12:00:06.946586624    129
2019-09-25 12:00:11.241553920    129
2019-09-25 12:00:15.536521216    128
Name: 11ed9a21-4371-5aa1-8085-b990869d0b51 (count), dtype: int64

### Multiple Series to DataFrames

If you would like to combine multiple series into a `pd.DataFrame`, the simplest way to do this using the manual transformer method is to use `pd.concat` which will join multiple series on their index. For example, to compose a phasor, you might do this as follows:

In [6]:
vphm = db.stream_from_uuid("11ed9a21-4371-5aa1-8085-b990869d0b51") # Northeast Carver 3 Ean VPHM
vpha = db.stream_from_uuid("8b696776-4a47-54b6-ae3b-4ff6bd218a3f") # Northeast Carver 3 Ean VPHA

START = "2019-09-25T12:00:00.000"
END = "2019-09-25T12:01:00.000"

df = pd.concat([
    points_to_series(vphm.values(START, END), name="VPHM"),
    points_to_series(vpha.values(START, END), name="VPHA"),
], axis=1)

df.head()

,VPHM,VPHA
2019-09-25 12:00:00.000000000,67837.398438,-75.875275
2019-09-25 12:00:00.033333100,67878.062500,-75.910896
2019-09-25 12:00:00.066666300,67845.203125,-75.969505
2019-09-25 12:00:00.100000200,67860.359375,-76.082619
2019-09-25 12:00:00.133333200,67881.507812,-76.084686


Note that `axis=1` is used to ensure the columns are joined instead of simply appending the rows together. This function relies on the index to align the columns; if there is a timestamp misalignment then `pd.concat` will fill in the missing column with `np.nan`. This can cause problems in some computations, so be sure to check if there are NaNs in your data as follows:

In [7]:
df.isna().sum()

VPHM    0
VPHA    0
dtype: int64

**Warning**: Always keep in mind that the tags and annotations sent to transformers must be in the same order as the columns in the data frames you compose. This will often lead to functions in your code that look like this:

```python
def myvalues(streams, start, end):
    df = pd.concat([
        points_to_series(stream.values(start, end), name=str(stream.uuid))
        for stream in streams
    ], axis=1)
    
    tags = [stream.tags() for stream in streams]
    annotations = [stream.annotations()[0] for stream in streams]
    
    return df, tags, annotations
```

However, if you find yourself doing this a lot, I would strongly recommend using the Conduit data flow API instead of the transformer API.

## Conversions

The conversions transformers take a stream as input and then apply a conversion to the data and metadata. These functions take a single stream as input and output a single stream as output.


In [8]:
# Test data for conversions
vphm = db.stream_from_uuid("11ed9a21-4371-5aa1-8085-b990869d0b51") # Northeast Carver 3 Ean VPHM
vpha = db.stream_from_uuid("8b696776-4a47-54b6-ae3b-4ff6bd218a3f") # Northeast Carver 3 Ean VPHA
iphm = db.stream_from_uuid("994c4914-1a7d-51d2-b7cc-8590787dc250") # Northeast Carver 3 Ia IPHM
ipha = db.stream_from_uuid("d699e645-09ce-51da-9e16-825d1771cc44") # Northeast Carver 3 Ia IPHA

START = "2019-09-25T12:00:00.000"
END = "2019-09-25T12:01:00.000"


def fetch_values(stream, start=START, end=END):
    series = points_to_series(stream.values(start, end), name=str(stream.uuid))
    return series, stream.tags(), stream.annotations()[0]

### Per Unit

In [9]:
help(per_unit)

Help on function per_unit in module conduit.transformers.conversions:

per_unit(series, tags, annotations, base_kv=None)
    Convert a magnitude stream into per-unit quantities.
    
    The Per Unit system is a unit-less domain that allows many common power
    engineering calculations to be carried out in a much simpler fashion. Voltage
    and current magnitudes are converted using different formulas that are
    determined by the specified stream units in the tags.
    
    Voltage is converted to Per Unit by dividing the volts by the Base KV.
    Base KV can be set to anything but is generally mapped to the voltage level of
    the network. For example a 500kV network has a Base KV of 500. Base KV is
    represented in Line-to-Line KV.
    
    Conversion of voltage magnitudes to per unit is applied differently based on
    the starting units. Voltage magnitudes are traditionally reported in
    line-to-neutral volts but are commonly displayed in line-to-line volts. The
    differ

In [10]:
series, tags, annotations = per_unit(*fetch_values(iphm), base_kv=500)
series.head()

2019-09-25 12:00:00.000000000    1.609680
2019-09-25 12:00:00.033333100    1.600887
2019-09-25 12:00:00.066666300    1.584741
2019-09-25 12:00:00.100000200    1.617867
2019-09-25 12:00:00.133333200    1.604629
Name: 994c4914-1a7d-51d2-b7cc-8590787dc250, dtype: float64

In [11]:
tags.pprint()

{'distiller': '',
 'ingress': 'dfr_72_80',
 'name': 'DFR!PMU-NOR2_237-PM40',
 'unit': 'Per Unit'}


In [12]:
annotations.pprint()

{'acronym': 'IPHM',
 'base_kv': 500,
 'description': 'Northeast DFR 2 CH 40: Carver 3 Ia',
 'devacronym': 'DFR!PMU-NOR2_237',
 'enabled': True,
 'id': '6e76eb5b-a352-4f16-97e7-22ae3b1d9325',
 'internal': True,
 'label': 'Northeast Carver 3 Ia',
 'measurement_type': 'Current Magnitude',
 'phase': 'A',
 'reference': 'DFR!PMU-NOR2_237-PM40',
 'type': 'I'}


In [13]:
series, tags, annotations = fetch_values(vphm)

# Update voltage units to line to neutral
tags[UNIT] = VOLTS_L_N

series, tags, annotations = per_unit(series, tags, annotations, base_kv=500)
series.head()

2019-09-25 12:00:00.000000000    0.234996
2019-09-25 12:00:00.033333100    0.235137
2019-09-25 12:00:00.066666300    0.235023
2019-09-25 12:00:00.100000200    0.235075
2019-09-25 12:00:00.133333200    0.235148
Name: 11ed9a21-4371-5aa1-8085-b990869d0b51, dtype: float64

### Line to Line

In [14]:
help(line_to_line)

Help on function line_to_line in module conduit.transformers.conversions:

line_to_line(series, tags, annotations, base_kv=None)
    Converts a voltage magnitude into Line-to-Line volts.
    
    Voltage magnitudes may begin as Per Unit, Line-to-Neutral or Line-to-Line.
    
    Parameters
    ----------
    series : pd.Series
        The input stream for the transformation
    
    tags : Tags
        Tags must contain the unit of the streams, tags are updated with the
        new stream units as VOLTS_L_L after successful computation.
    
    annotations : Annotations
        May contain the ``base_kv`` parameter so it does not have to be specified.
    
    base_kv : float, default: None
        The Base KV of the stream for Per-Unit conversion. If None, the value is
        discovered from the annotations.
    
    Returns
    -------
    series : pd.Series
        The output stream following the transformation
    
    tags : Tags
        Tags updated with the new units
    
    

In [15]:
# Converting per-unit to line to line
# Note that base_kv is in the annotations now!
series, tags, annotations = line_to_line(series, tags, annotations)
series.head()

2019-09-25 12:00:00.000000000    117497.820747
2019-09-25 12:00:00.033333100    117568.252969
2019-09-25 12:00:00.066666300    117511.338862
2019-09-25 12:00:00.100000200    117537.590257
2019-09-25 12:00:00.133333200    117574.220426
Name: 11ed9a21-4371-5aa1-8085-b990869d0b51, dtype: float64

In [16]:
tags.pprint()

{'distiller': '',
 'ingress': 'dfr_72_80',
 'name': 'DFR!PMU-NOR2_237-PM37',
 'unit': 'Voltage Line-to-Line'}


### Line to Neutral

In [17]:
help(line_to_neutral)

Help on function line_to_neutral in module conduit.transformers.conversions:

line_to_neutral(series, tags, annotations, base_kv=None)
    Converts a voltage magnitude into Line-to-Neutral volts.
    
    Voltage magnitudes may begin as Per Unit, Line-to-Neutral or Line-to-Line.
    
    Parameters
    ----------
    series : pd.Series
        The input stream for the transformation
    
    tags : Tags
        Tags must contain the unit of the streams, tags are updated with the
        new stream units as VOLTS_L_N after successful computation.
    
    annotations : Annotations
        May contain the ``base_kv`` parameter so it does not have to be specified.
    
    base_kv : float, default: None
        The Base KV of the stream for Per-Unit conversion. If None, the value is
        discovered from the annotations.
    
    Returns
    -------
    series : pd.Series
        The output stream following the transformation
    
    tags : Tags
        Tags updated with the new units


In [18]:
# Hopefully this equals our original data!
# Note that base_kv is in the annotations now!
series, tags, annotations = line_to_neutral(series, tags, annotations)
series.head()

2019-09-25 12:00:00.000000000    67837.398437
2019-09-25 12:00:00.033333100    67878.062500
2019-09-25 12:00:00.066666300    67845.203125
2019-09-25 12:00:00.100000200    67860.359375
2019-09-25 12:00:00.133333200    67881.507812
Name: 11ed9a21-4371-5aa1-8085-b990869d0b51, dtype: float64

### Amps

If we need to convert IPHM per-unit back to amps

In [19]:
help(amps)

Help on function amps in module conduit.transformers.conversions:

amps(series, tags, annotations, base_kv=None)
    Converts a current magnitude from Per Unit into Amps.
    
    Parameters
    ----------
    series : pd.Series
        The input stream for the transformation
    
    tags : Tags
        Tags must contain the unit of the streams, tags are updated with the
        new stream units as AMPS after successful computation.
    
    annotations : Annotations
        May contain the ``base_kv`` parameter so it does not have to be specified.
    
    base_kv : float, default: None
        The Base KV of the stream for Per-Unit conversion. If None, the value is
        discovered from the annotations.
    
    Returns
    -------
    series : pd.Series
        The output stream following the transformation
    
    tags : Tags
        Tags updated with the new units
    
    annotations : Annotations
        Annotations updated with the base_kv and stream type.



### Calibrate

In [20]:
help(calibrate)

Help on function calibrate in module conduit.transformers.conversions:

calibrate(series, tags, annotations, rcf=None, pacf=None)
    Calibrates phasor measurement components by applying a multiplicative correction
    (ratio correction factor - RCF) for phasor magnitudes and an additive correction
    factor (phase angle correction factor - PACF) to phasor angles.
    
    Parameters
    ----------
    series : pd.Series
        The input stream for the transformation
    
    tags : Tags
        Tags must contain the unit of the streams.
    
    annotations : Annotations
        May contain the ``rcf`` and ``pacf`` parameters so they do not have to be
        specified directly.
    
    rcf : float, default: None
        The Ratio Correction Factor, a multiplicative correction factor for phasor
        magnitudes. If None, the value is discovered from the annotations.
    
    pacf : float, default: None
        The Phase Angle Correction Factor, an additive correction factor for p

In [21]:
series, tags, annotations = calibrate(*fetch_values(vphm), rcf=0.05)
series.head()

2019-09-25 12:00:00.000000000    3391.869922
2019-09-25 12:00:00.033333100    3393.903125
2019-09-25 12:00:00.066666300    3392.260156
2019-09-25 12:00:00.100000200    3393.017969
2019-09-25 12:00:00.133333200    3394.075391
Name: 11ed9a21-4371-5aa1-8085-b990869d0b51, dtype: float64

In [22]:
series, tags, annotations = calibrate(*fetch_values(vpha), pacf=0.12)
series.head()

2019-09-25 12:00:00.000000000   -75.755275
2019-09-25 12:00:00.033333100   -75.790896
2019-09-25 12:00:00.066666300   -75.849505
2019-09-25 12:00:00.100000200   -75.962619
2019-09-25 12:00:00.133333200   -75.964686
Name: 8b696776-4a47-54b6-ae3b-4ff6bd218a3f, dtype: float64

In [23]:
tags.pprint()

{'distiller': '',
 'ingress': 'dfr_72_80',
 'name': 'DFR!PMU-NOR2_237-PA37',
 'unit': 'VPHA'}


In [24]:
annotations.pprint()

{'acronym': 'VPHA',
 'description': 'Northeast DFR 2 CH 37: Carver 3 Ean',
 'devacronym': 'DFR!PMU-NOR2_237',
 'enabled': True,
 'id': '24b074cc-20e7-477a-a7a3-8cf02459c487',
 'internal': True,
 'label': 'Northeast Carver 3 Ean',
 'measurement_type': 'Voltage Angle',
 'pacf': 0.12,
 'phase': 'A',
 'reference': 'DFR!PMU-NOR2_237-PA37',
 'type': 'V'}


### Radians

In [25]:
help(radians)

Help on function radians in module conduit.transformers.conversions:

radians(series, tags, annotations)
    Converts phasor angle into radians.
    
    Parameters
    ----------
    series : pd.Series
        The input stream for the transformation
    
    tags : Tags
        Tags must contain the unit of the streams, tags are updated with the
        new stream units as RADIANS after successful computation.
    
    annotations : Annotations
        Annotations belonging to the stream
    
    Returns
    -------
    series : pd.Series
        The output stream following the transformation
    
    tags : Tags
        Tags updated with the new units
    
    annotations : Annotations
        Annotations belonging to the stream



In [26]:
series, tags, annotations = radians(*fetch_values(ipha))
series.head()

2019-09-25 12:00:00.000000000   -1.639941
2019-09-25 12:00:00.033333100   -1.642482
2019-09-25 12:00:00.066666300   -1.632414
2019-09-25 12:00:00.100000200   -1.649094
2019-09-25 12:00:00.133333200   -1.641189
Name: d699e645-09ce-51da-9e16-825d1771cc44, dtype: float64

### Degrees

In [27]:
help(degrees)

Help on function degrees in module conduit.transformers.conversions:

degrees(series, tags, annotations)
    Converts phasor angle into degrees.
    
    Parameters
    ----------
    series : pd.Series
        The input stream for the transformation
    
    tags : Tags
        Tags must contain the unit of the streams, tags are updated with the
        new stream units as DEGREES after successful computation.
    
    annotations : Annotations
        Annotations belonging to the stream
    
    Returns
    -------
    series : pd.Series
        The output stream following the transformation
    
    tags : Tags
        Tags updated with the new units
    
    annotations : Annotations
        Annotations belonging to the stream



In [28]:
series, tags, annotations = degrees(series, tags, annotations)
series.head()

2019-09-25 12:00:00.000000000   -93.961723
2019-09-25 12:00:00.033333100   -94.107292
2019-09-25 12:00:00.066666300   -93.530457
2019-09-25 12:00:00.100000200   -94.486115
2019-09-25 12:00:00.133333200   -94.033226
Name: d699e645-09ce-51da-9e16-825d1771cc44, dtype: float64

## Phasors

The phasor transformers operate on a data frame that is composed of two streams - a magnitude and an angle. The order of the streams does not matter so long as the tags and annotations are in the correct order. 

In [29]:
def fetch_values(*streams, start=START, end=END):
    df = pd.concat([
        points_to_series(stream.values(start, end), name=str(stream.uuid))
        for stream in streams
    ], axis=1)
    
    tags, annotations = [], []
    for stream in streams:
        tags.append(stream.tags())
        annotations.append(stream.annotations()[0])
    
    return df, tags, annotations

### Real

In [30]:
help(real)

Help on function real in module conduit.transformers.phasor:

real(df, tags, annotations)
    Converts the two component streams of a Phasor (Magnitude and Angle) into the
    Real part of the Complex Phasor. Handles angles in degrees or radians.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with two columns: one for magnitude and one for angle.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series with the real component of the complex phasor
    
    tags : Tags
        A single tagset with the units of the magnitude stream
    
    annotations : Annotations
        A single annotationset with the new stream type



In [31]:
series, tags, annotations = real(*fetch_values(vphm, vpha))
series.head()

2019-09-25 12:00:00.000000000    16554.599557
2019-09-25 12:00:00.033333100    16523.594812
2019-09-25 12:00:00.066666300    16448.274655
2019-09-25 12:00:00.100000200    16321.943873
2019-09-25 12:00:00.133333200    16324.652888
Name: Phasor (Real), dtype: float64

### Imaginary

In [32]:
help(imaginary)

Help on function imaginary in module conduit.transformers.phasor:

imaginary(df, tags, annotations)
    Converts the two component streams of a Phasor (Magnitude and Angle) into the
    Imaginary part of the Complex Phasor. Handles angles in degrees or radians.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with two columns: one for magnitude and one for angle.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series with the imaginary component of the complex phasor
    
    tags : Tags
        A single tagset with the units of the magnitude stream
    
    annotations : Annotations
        A single annotationset with the new stream type



In [33]:
series, tags, annotations = imaginary(*fetch_values(vphm, vpha))
series.head()

2019-09-25 12:00:00.000000000   -65786.456511
2019-09-25 12:00:00.033333100   -65836.176858
2019-09-25 12:00:00.066666300   -65821.165653
2019-09-25 12:00:00.100000200   -65868.220886
2019-09-25 12:00:00.133333200   -65889.337612
Name: Phasor (Imag), dtype: float64

### Complex Phasor

In [34]:
help(complex_phasor)

Help on function complex_phasor in module conduit.transformers.phasor:

complex_phasor(df, tags, annotations)
    Converts the two component streams of a Phasor (Magnitude and Angle) into a
    single complex valued stream. Handles angles in degrees or radians.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with two columns: one for magnitude and one for angle.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series with the complex phasor
    
    tags : Tags
        A single tagset with the units of the magnitude stream
    
    annotations : Annotations
        A single annotationset with the new stream type



In [35]:
series, tags, annotations = complex_phasor(*fetch_values(vphm, vpha))
series.head()

2019-09-25 12:00:00.000000000    (16554.599557305664-65786.45651093683j)
2019-09-25 12:00:00.033333100    (16523.594812107407-65836.17685770646j)
2019-09-25 12:00:00.066666300    (16448.274654954985-65821.16565321277j)
2019-09-25 12:00:00.100000200     (16321.94387345219-65868.22088606938j)
2019-09-25 12:00:00.133333200    (16324.652887704258-65889.33761235172j)
Name: Phasor (Complex), dtype: complex128

In [36]:
tags.pprint()

{'distiller': '',
 'ingress': 'dfr_72_80',
 'name': 'Phasor (Complex)',
 'unit': 'VPHM'}


In [37]:
annotations.pprint()

{'acronym': {'VPHA', 'VPHM'},
 'description': 'Northeast DFR 2 CH 37: Carver 3 Ean',
 'devacronym': 'DFR!PMU-NOR2_237',
 'enabled': True,
 'id': {'24b074cc-20e7-477a-a7a3-8cf02459c487',
        '8455c8a7-dc41-4a62-b693-5c50cc2b7198'},
 'internal': True,
 'label': 'Northeast Carver 3 Ean',
 'measurement_type': 'Phasor (Complex)',
 'phase': 'A',
 'reference': {'DFR!PMU-NOR2_237-PA37', 'DFR!PMU-NOR2_237-PM37'},
 'type': 'V'}


## Phasor Pairs

The phasor pair transformers operate on a data frame that is composed of four streams - both the voltage and the current magnitude and angles. The order of the streams does not matter so long as the tags and annotations are in the correct order. 

### Complex Power

In [38]:
help(complex_power)

Help on function complex_power in module conduit.transformers.phasor:

complex_power(df, tags, annotations)
    This computes the complex power (P + jQ) in Per Unit or MW+jMVAR based on
    the input units.
    
    Formula: complex_voltage * conj(complex_current)
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with four columns: voltage and current angle and magnitudes.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of complex power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [39]:
series, tags, annotations = complex_power(*fetch_values(vphm, iphm, vpha, ipha))
series.head()

2019-09-25 12:00:00.000000000    (11985909.224562028+3914461.544905271j)
2019-09-25 12:00:00.033333100    (11920083.333385102+3918293.133185567j)
2019-09-25 12:00:00.066666300     (11836418.09083853+3745856.287387613j)
2019-09-25 12:00:00.100000200      (12028983.02103727+4002325.21684728j)
2019-09-25 12:00:00.133333200    (11965427.894051684+3875925.331914663j)
Name: Complex Power, dtype: complex128

In [40]:
tags.pprint()

{'distiller': '',
 'ingress': 'dfr_72_80',
 'name': 'Complex Power',
 'unit': 'MW + jMVAR'}


In [41]:
annotations.pprint()

{'acronym': {'VPHA', 'IPHA', 'VPHM', 'IPHM'},
 'description': {'Northeast DFR 2 CH 37: Carver 3 Ean',
                 'Northeast DFR 2 CH 40: Carver 3 Ia'},
 'devacronym': 'DFR!PMU-NOR2_237',
 'enabled': True,
 'id': {'24b074cc-20e7-477a-a7a3-8cf02459c487',
        '6e76eb5b-a352-4f16-97e7-22ae3b1d9325',
        '8455c8a7-dc41-4a62-b693-5c50cc2b7198',
        'a4cf92f0-9a7a-4023-8cc5-f1a3f0c22c5a'},
 'internal': True,
 'label': {'Northeast Carver 3 Ia', 'Northeast Carver 3 Ean'},
 'measurement_type': 'Complex Power',
 'phase': 'A',
 'reference': {'DFR!PMU-NOR2_237-PA37',
               'DFR!PMU-NOR2_237-PA40',
               'DFR!PMU-NOR2_237-PM37',
               'DFR!PMU-NOR2_237-PM40'},
 'type': {'V', 'I'}}


### Real Power

In [42]:
help(real_power)

Help on function real_power in module conduit.transformers.phasor:

real_power(df, tags, annotations)
    Computes the real power (P) in Per Unit or MW based on the input units.
    
    Formula: (complex_voltage * conj(complex_current)).real
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with four columns: voltage and current angle and magnitudes.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of real power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [43]:
series, tags, annotations = real_power(*fetch_values(vphm, iphm, vpha, ipha))
series.head()

2019-09-25 12:00:00.000000000    1.198591e+07
2019-09-25 12:00:00.033333100    1.192008e+07
2019-09-25 12:00:00.066666300    1.183642e+07
2019-09-25 12:00:00.100000200    1.202898e+07
2019-09-25 12:00:00.133333200    1.196543e+07
Name: Real Power, dtype: float64

### Reactive Power

In [44]:
help(reactive_power)

Help on function reactive_power in module conduit.transformers.phasor:

reactive_power(df, tags, annotations)
    This computes the reactive power (Q) in Per Unit or MVAR based on the input units.
    
    Formula: (complex_voltage * conj(complex_current)).imag
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with four columns: voltage and current angle and magnitudes.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of reactive power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [45]:
series, tags, annotations = reactive_power(*fetch_values(vphm, iphm, vpha, ipha))
series.head()

2019-09-25 12:00:00.000000000    3.914462e+06
2019-09-25 12:00:00.033333100    3.918293e+06
2019-09-25 12:00:00.066666300    3.745856e+06
2019-09-25 12:00:00.100000200    4.002325e+06
2019-09-25 12:00:00.133333200    3.875925e+06
Name: Reactive Power, dtype: float64

### Apparent Power

In [46]:
help(apparent_power)

Help on function apparent_power in module conduit.transformers.phasor:

apparent_power(df, tags, annotations)
    This computes the apparent power (S) in Per Unit or MVA based on the input units.
    
    Formula: abs(complex_voltage * conj(complex_current))
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with four columns: voltage and current angle and magnitudes.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of apparent power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [47]:
series, tags, annotations = apparent_power(*fetch_values(vphm, iphm, vpha, ipha))
series.head()

2019-09-25 12:00:00.000000000    1.260893e+07
2019-09-25 12:00:00.033333100    1.254757e+07
2019-09-25 12:00:00.066666300    1.241500e+07
2019-09-25 12:00:00.100000200    1.267734e+07
2019-09-25 12:00:00.133333200    1.257753e+07
Name: Apparent Power, dtype: float64

### Power Factor

In [48]:
help(power_factor)

Help on function power_factor in module conduit.transformers.phasor:

power_factor(df, tags, annotations)
    Computes the power factor.
    
    Formula: real_power() / apparent_power()
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with four columns: voltage and current angle and magnitudes.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of power factor values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [49]:
series, tags, annotations = power_factor(*fetch_values(vphm, iphm, vpha, ipha))
series.head()

2019-09-25 12:00:00.000000000    0.950589
2019-09-25 12:00:00.033333100    0.949992
2019-09-25 12:00:00.066666300    0.953397
2019-09-25 12:00:00.100000200    0.948857
2019-09-25 12:00:00.133333200    0.951334
Name: Power Factor, dtype: float64

## Phasor Groups

The phasor group transformers operate on a data frame that is composed of six streams - magnitude and angles for all three phases of either voltage or current. The order of the streams does not matter so long as the tags and annotations are in the correct order.

In [50]:
vamag = db.stream_from_uuid("11ed9a21-4371-5aa1-8085-b990869d0b51") # Northeast Carver 3 Ean A VPHM
vaang = db.stream_from_uuid("8b696776-4a47-54b6-ae3b-4ff6bd218a3f") # Northeast Carver 3 Ean A VPHA
vbmag = db.stream_from_uuid("ba69274f-1ddf-5f02-a750-e7b30bc3a0ea") # Northeast Carver 3 Ebn B VPHM
vbang = db.stream_from_uuid("a35ecf3e-1d74-5fe0-9894-44cc133c0e38") # Northeast Carver 3 Ebn B VPHA
vcmag = db.stream_from_uuid("30866085-71f4-588e-a548-7c58255aaf2f") # Northeast Carver 3 Ecn C VPHM
vcang = db.stream_from_uuid("bcf12c7c-8db9-52c3-b03c-3126ed857dbe") # Northeast Carver 3 Ecn C VPHA

iamag = db.stream_from_uuid("994c4914-1a7d-51d2-b7cc-8590787dc250") # Northeast Carver 3 Ia  A IPHM
iaang = db.stream_from_uuid("d699e645-09ce-51da-9e16-825d1771cc44") # Northeast Carver 3 Ia  A IPHA
ibmag = db.stream_from_uuid("d31de3b9-3fa1-50c9-b321-c3fc0d676f36") # Northeast Carver 3 Ib  B IPHM
ibang = db.stream_from_uuid("b3dffba0-8fb1-53ef-956a-2de09911c6b4") # Northeast Carver 3 Ib  B IPHA
icmag = db.stream_from_uuid("226ed982-55bb-56db-9df0-ad4987eebca4") # Northeast Carver 3 Ic  C IPHM
icang = db.stream_from_uuid("70c443f5-3ee0-5e44-a72e-d9898b002114") # Northeast Carver 3 Ic  C IPHA

### Complex Phasor Group

In [51]:
help(complex_phasor_group)

Help on function complex_phasor_group in module conduit.transformers.phasor:

complex_phasor_group(df, tags, annotations)
    Returns the phasor group as complex phasors for each power phase.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with six columns: angle and magnitudes for three phases.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    data : pd.DataFrame
        A DataFrame with a complex phasor for each power phase
    
    tags : TagsGroup
        Tags describing each column (power phase) in the output
    
    annotations : AnnotationsGroup
        Annotations describing each column (power phase) in the output



In [52]:
df, tags, annotations = complex_phasor_group(*fetch_values(vamag, vbmag, vcmag, vaang, vbang, vcang))
df.head()

,Phase A,Phase B,Phase C
2019-09-25 12:00:00.000000000,(16554.599557305664-65786.45651093683j),(-65549.85809076928+18478.529450977243j),(48961.09892911847+47339.69245258796j)
2019-09-25 12:00:00.033333100,(16523.594812107407-65836.17685770646j),(-65542.07656572085+18538.83153070996j),(48976.035527156295+47341.57447823345j)
2019-09-25 12:00:00.066666300,(16448.274654954985-65821.16565321277j),(-65517.785694548795+18568.015351667025j),(49043.13305288382+47283.037157749306j)
2019-09-25 12:00:00.100000200,(16321.94387345219-65868.22088606938j),(-65490.0252807664+18650.037252234473j),(49099.46017793011+47243.04700471929j)
2019-09-25 12:00:00.133333200,(16324.652887704258-65889.33761235172j),(-65463.963648283694+18759.98081494591j),(49085.63863773546+47154.38227058879j)


In [53]:
tags.pprint()

[{'distiller': '', 'ingress': 'dfr_72_80', 'name': 'Phase A', 'unit': 'VPHM'},
 {'distiller': '', 'ingress': 'dfr_72_80', 'name': 'Phase B', 'unit': 'VPHM'},
 {'distiller': '', 'ingress': 'dfr_72_80', 'name': 'Phase C', 'unit': 'VPHM'}]


In [54]:
annotations.pprint()

[{'acronym': {'VPHA', 'VPHM'},
  'description': 'Northeast DFR 2 CH 37: Carver 3 Ean',
  'devacronym': 'DFR!PMU-NOR2_237',
  'enabled': True,
  'id': {'24b074cc-20e7-477a-a7a3-8cf02459c487',
         '8455c8a7-dc41-4a62-b693-5c50cc2b7198'},
  'internal': True,
  'label': 'Northeast Carver 3 Ean',
  'measurement_type': 'Phasor (Complex)',
  'phase': 'A',
  'reference': {'DFR!PMU-NOR2_237-PA37', 'DFR!PMU-NOR2_237-PM37'},
  'type': 'V'},
 {'acronym': {'VPHA', 'VPHM'},
  'description': 'Northeast DFR 2 CH 38: Carver 3 Ebn',
  'devacronym': 'DFR!PMU-NOR2_237',
  'enabled': True,
  'id': {'8d0ac3a5-0077-42ab-81d0-1b7f600bb2e1',
         'ecd9e83b-76b4-4a62-9007-5d46e56fa388'},
  'internal': True,
  'label': 'Northeast Carver 3 Ebn',
  'measurement_type': 'Phasor (Complex)',
  'phase': 'B',
  'reference': {'DFR!PMU-NOR2_237-PA38', 'DFR!PMU-NOR2_237-PM38'},
  'type': 'V'},
 {'acronym': {'VPHA', 'VPHM'},
  'description': 'Northeast DFR 2 CH 39: Carver 3 Ecn',
  'devacronym': 'DFR!PMU-NOR2_237',

### Sequence Components

In [55]:
help(sequence_components)

Help on function sequence_components in module conduit.transformers.phasor:

sequence_components(df, tags, annotations)
    Formula is here: https://en.wikipedia.org/wiki/Symmetrical_components
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with six columns: angle and magnitudes for three phases.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    data : pd.DataFrame
        A DataFrame with all three sequence components over time
    
    tags : TagsGroup
        Tags describing each column (sequence component) in the output
    
    annotations : AnnotationsGroup
        Annotations describing each column (sequence component) in the output



In [56]:
df, tags, annotations = sequence_components(*fetch_values(vamag, vbmag, vcmag, vaang, vbang, vcang))
df.head()

,0,1,2
2019-09-25 12:00:00.000000000,(-11.386534781716666+10.588464209458227j),(16614.493160096477-65954.98841787007j),(-48.50706800910848+157.94344272378294j)
2019-09-25 12:00:00.033333100,(-14.148742152385239+14.743050412313702j),(16583.507474206017-65983.99137601457j),(-45.76391994623797+133.07146789578715j)
2019-09-25 12:00:00.066666300,(-8.792662236662483+9.96228540118803j),(16517.846443359595-65986.45260802751j),(-60.779126167957045+155.32466941355352j)
2019-09-25 12:00:00.100000200,(-22.873743128033915+8.287790294792405j),(16426.499747059443-66017.38947612165j),(-81.682130479232+140.8807997574792j)
2019-09-25 12:00:00.133333200,(-17.890707614659426+8.341824394325764j),(16368.029459606303-66016.46157607189j),(-25.485864287390235+118.78213932583822j)


### Zero Sequence

In [57]:
help(zero_sequence)

Help on function zero_sequence in module conduit.transformers.phasor:

zero_sequence(df, tags, annotations)
    Formula is (A + B + C) / 3.
    This is phasor math in complex plane.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with six columns: angle and magnitudes for three phases.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of the complex zero sequence component over time.
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [58]:
series, tags, annotations = zero_sequence(*fetch_values(iamag, ibmag, icmag, iaang, ibang, icang))
series.head()

2019-09-25 12:00:00.000000000      (2.23949310621893+2.6462203519525738j)
2019-09-25 12:00:00.033333100     (1.5695822672592876+3.684455236186873j)
2019-09-25 12:00:00.066666300    (2.1526488740688023+3.4309302394670214j)
2019-09-25 12:00:00.100000200     (1.7474029518463965+2.557173334397087j)
2019-09-25 12:00:00.133333200     (1.911485538275182+2.6919245012804867j)
Name: 0, dtype: complex128

### Positive Sequence

In [59]:
help(positive_sequence)

Help on function positive_sequence in module conduit.transformers.phasor:

positive_sequence(df, tags, annotations)
    Formula is (A + B * 1<120 + C * 1<240) / 3
    ALPHA and ALPHA_2 are unit vectors at 120 degrees and -120 degrees, respectively.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with six columns: angle and magnitudes for three phases.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of the complex positive sequence component over time.
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [60]:
series, tags, annotations = positive_sequence(*fetch_values(iamag, ibmag, icmag, iaang, ibang, icang))
series.head()

2019-09-25 12:00:00.000000000    (-16.403419329825557-180.23570452358334j)
2019-09-25 12:00:00.033333100      (-16.20830021993057-180.9226513069945j)
2019-09-25 12:00:00.066666300    (-15.565139615876918-179.44080903244947j)
2019-09-25 12:00:00.100000200    (-17.485074351805768-181.04938818482356j)
2019-09-25 12:00:00.133333200     (-16.227252795516353-179.4508411644463j)
Name: 1, dtype: complex128

### Negative Sequence

In [61]:
help(negative_sequence)

Help on function negative_sequence in module conduit.transformers.phasor:

negative_sequence(df, tags, annotations)
    Formula is (A + B * 1<240 + C * 1<120) / 3
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with six columns: angle and magnitudes for three phases.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of the complex negative sequence component over time.
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [62]:
series, tags, annotations = negative_sequence(*fetch_values(iamag, ibmag, icmag, iaang, ibang, icang))
series.head()

2019-09-25 12:00:00.000000000    (1.3221729271500255-7.836209123927849j)
2019-09-25 12:00:00.033333100    (1.3986265249328274-7.141544958935862j)
2019-09-25 12:00:00.066666300     (2.144124093195633-6.632940748163965j)
2019-09-25 12:00:00.100000200    (1.1254583483192278-7.750598107157808j)
2019-09-25 12:00:00.133333200     (1.2836468938047574-8.06875313408868j)
Name: 2, dtype: complex128

## Phasor Group Pairs

The phasor group pair transformers operate on a data frame that is composed of 12 streams - magnitude and angles for all three phases of both voltage and current. The order of the streams does not matter so long as the tags and annotations are in the correct order.

### Three Phase Complex Power

In [63]:
help(three_phase_complex_power)

Help on function three_phase_complex_power in module conduit.transformers.phasor:

three_phase_complex_power(df, tags, annotations)
    Computes the complex power (P + jQ) summed across all three phases.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with 12 columns: voltage and current angles and magnitudes for
        three phases, A, B, and C.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of three phase complex power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [64]:
series, tags, annotations = three_phase_complex_power(*fetch_values(
    vamag, iamag, vbmag, ibmag, vcmag, icmag, vaang, iaang, vbang, ibang, vcang, icang
))
series.head()

2019-09-25 12:00:00.000000000     (34840830.03853895+12228884.319036357j)
2019-09-25 12:00:00.033333100     (35004677.89714943+12209265.135494957j)
2019-09-25 12:00:00.066666300     (34747243.59058613+11973136.740621518j)
2019-09-25 12:00:00.100000200    (34991960.364504814+12383775.411200736j)
2019-09-25 12:00:00.133333200    (34740295.583532214+12025600.218836717j)
dtype: complex128

In [65]:
tags.pprint()

{'distiller': '',
 'ingress': 'dfr_72_80',
 'name': 'Complex Power',
 'unit': 'MW + jMVAR'}


In [66]:
annotations.pprint()

{'acronym': {'VPHA', 'IPHA', 'VPHM', 'IPHM'},
 'description': set(),
 'devacronym': 'DFR!PMU-NOR2_237',
 'enabled': True,
 'id': set(),
 'internal': True,
 'label': set(),
 'measurement_type': 'Complex Power',
 'phase': {'A', 'B', 'C'},
 'reference': set(),
 'type': {'V', 'I'}}


### Three Phase MW

In [67]:
help(three_phase_mw)

Help on function three_phase_mw in module conduit.transformers.phasor:

three_phase_mw(df, tags, annotations)
    Computes the real power (P) summed across all three phases.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with 12 columns: voltage and current angles and magnitudes for
        three phases, A, B, and C.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of three phase real power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [68]:
series, tags, annotations = three_phase_mw(*fetch_values(
    vamag, iamag, vbmag, ibmag, vcmag, icmag, vaang, iaang, vbang, ibang, vcang, icang
))
series.head()

2019-09-25 12:00:00.000000000    3.484083e+07
2019-09-25 12:00:00.033333100    3.500468e+07
2019-09-25 12:00:00.066666300    3.474724e+07
2019-09-25 12:00:00.100000200    3.499196e+07
2019-09-25 12:00:00.133333200    3.474030e+07
dtype: float64

### Three Phase MVAR

In [69]:
help(three_phase_mvar)

Help on function three_phase_mvar in module conduit.transformers.phasor:

three_phase_mvar(df, tags, annotations)
    Computes the reactive power (Q) summed across all three phases.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with 12 columns: voltage and current angles and magnitudes for
        three phases, A, B, and C.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of three phase reactive power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [70]:
series, tags, annotations = three_phase_mvar(*fetch_values(
    vamag, iamag, vbmag, ibmag, vcmag, icmag, vaang, iaang, vbang, ibang, vcang, icang
))
series.head()

2019-09-25 12:00:00.000000000    1.222888e+07
2019-09-25 12:00:00.033333100    1.220927e+07
2019-09-25 12:00:00.066666300    1.197314e+07
2019-09-25 12:00:00.100000200    1.238378e+07
2019-09-25 12:00:00.133333200    1.202560e+07
dtype: float64

### Three Phase MVA

In [71]:
help(three_phase_mva)

Help on function three_phase_mva in module conduit.transformers.phasor:

three_phase_mva(df, tags, annotations)
    Computes the apparent power (S) summed across all three phases.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with 12 columns: voltage and current angles and magnitudes for
        three phases, A, B, and C.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of three phase apparent power values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [72]:
series, tags, annotations = three_phase_mva(*fetch_values(
    vamag, iamag, vbmag, ibmag, vcmag, icmag, vaang, iaang, vbang, ibang, vcang, icang
))
series.head()

2019-09-25 12:00:00.000000000    3.696332e+07
2019-09-25 12:00:00.033333100    3.711091e+07
2019-09-25 12:00:00.066666300    3.678926e+07
2019-09-25 12:00:00.100000200    3.715465e+07
2019-09-25 12:00:00.133333200    3.680171e+07
dtype: float64

### Three Phase Power Factor

In [74]:
help(three_phase_power_factor)

Help on function three_phase_power_factor in module conduit.transformers.phasor:

three_phase_power_factor(df, tags, annotations)
    Computes the power factor summed across all three phases.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input data with 12 columns: voltage and current angles and magnitudes for
        three phases, A, B, and C.
    
    tags : TagsGroup
        A collection of tags that describe each column in the input data.
    
    annotations : AnnotationsGroup
        A collection of annotations that describe each column in the input data.
    
    Returns
    -------
    series : pd.Series
        A single series of three phase power factor values over time
    
    tags : Tags
        A single tagset with the units based on the input
    
    annotations : Annotations
        A single annotationset with the new stream type



In [75]:
series, tags, annotations = three_phase_power_factor(*fetch_values(
    vamag, iamag, vbmag, ibmag, vcmag, icmag, vaang, iaang, vbang, ibang, vcang, icang
))
series.head()

2019-09-25 12:00:00.000000000    2.827479
2019-09-25 12:00:00.033333100    2.829742
2019-09-25 12:00:00.066666300    2.833472
2019-09-25 12:00:00.100000200    2.825192
2019-09-25 12:00:00.133333200    2.831764
dtype: float64